# Class

### How to setup your local environment

#### Install the libraries needed

To install the OpenAI Python library and the rest of the dependencies needed execute this on the console:
```
conda install -r requirements.txt
```

#### Setup your OpenAI API key

##### First option: Setup your OpenAI API key as environment variable

The steps to do this are different depending on your operating system but the first step is the same to get your API key from the OpenAI dashboard:

1. You need to be logged in to your OpenAI account and then go to https://platform.openai.com/and create a new project
2. Click on your project and then on "API keys"
3. Click on "Create API key"
4. Copy the API key

* Now you can set the `OPENAI_API_KEY` environment variable to your API key and depending on your operating system you can do this by running the following command in the console:

**On Mac**

```
export OPENAI_API_KEY='sk-proj-...your key here...'
```

**On Windows**
```
$env:OPENAI_API_KEY='sk-proj-...your key here...'
```

##### Second option: Setup your OpenAI API key in a .yaml file (i used this one)

The steps to do this are:

1. Create a new file named `config.yaml` in the root of your project
2. Add the following content to the file:
```
api_credentials:
  openai:
    api_key: "API_KEY_HERE"
    organization: "your organization id"
```

### Lesson 1: LLMs , the chat format and tokens

In [1]:
import os
import yaml
from utils import get_completion_and_token_count,get_openai_client,check_moderation
# Load the API key from config.yaml
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
api_key = config['api_credentials']['openai']['api_key']
# Create the OpenAI client with the API key
client = get_openai_client(api_key=api_key)

In [2]:
prompt =  [  
{'role':'system',
 'content':"You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."},    
{'role':'user',
 'content':"write me a story about a happy carrot"},
] 
content, token_dict= get_completion_and_token_count(client,prompt,model="gpt-4o-mini",temperature=0.7,max_tokens=100)
print(content)
token_dict

In a garden so bright, under sun’s golden light, lived a carrot named Charlie, who danced with delight!


{'prompt_tokens': 42, 'completion_tokens': 23, 'total_tokens': 65}

### Lesson 2: Classification

In [3]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I want you to delete my profile and all of my user data"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
content, token_dict=get_completion_and_token_count(client,messages,model="gpt-4o-mini",temperature=0,max_tokens=200)
print(content)
token_dict

{
  "primary": "Account Management",
  "secondary": "Close account"
}


{'prompt_tokens': 163, 'completion_tokens': 18, 'total_tokens': 181}

### Lesson 3: Moderation

To use the Moderation API with versions of the openai library that are >= 1.0.0, the usage of the API may change slightly from older versions, but the concept is similar.

1. You need to have whisper-1 not blocked on the Model usage policy selected for the project you are using, otherwise it will throw an error.(in my case i blocked all the models except whisper-1 and gpt-4o-mini-2024-07-18)
2. You need to use the `client.moderations.create` method instead of the `openai.moderations.create` method.
3. select the "text-moderation-stable" model

In [5]:
models = client.models.list()
# Iterate through the models and print their IDs
for model in models.data:
    print(model.id)

whisper-1
gpt-4o-mini-2024-07-18


In [4]:
prompt = "I want to kill you"
check_moderation(client,prompt,model="text-moderation-stable")


Moderation response: ModerationCreateResponse(id='modr-A8SWDlTzYpghBXCPNl7SKMz0uAe4O', model='text-moderation-007', results=[Moderation(categories=Categories(harassment=True, harassment_threatening=True, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=True, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=True), category_scores=CategoryScores(harassment=0.7041232585906982, harassment_threatening=0.6778415441513062, hate=0.002266854979097843, hate_threatening=0.0002044593566097319, self_harm=2.0691999452537857e-05, self_harm_instructions=3.1496071528636094e-07, self_harm_intent=5.250821413937956e-05, sexual=5.0045597163261846e-05, sexual_minors=9.526918631763692e-08, violence=0.9961150884628296, violence_graphic=1.00255711004138e-05, self-harm=2.0691